Вопросы:
 - почему результат меняется при каждом запуске, хотя стоит seed=0 при инициализации весов? Что нужно ещё добавить?
 - объяснить рез-тат для модели на искаженных данных (val_acc vs. train_acc)
 - augmentation не помогает улучшить, а только ухудшает рез-тат. Почему?

# Convolutional Neural Networks: Application

Here we will try a CNN model on the SIGNS dataset. The SIGNS dataset is a collection of 6 signs representing numbers from 0 to 5.

<img src="images/SIGNS.png" style="width:800px;height:300px;">

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from cnn_utils import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import datetime

%matplotlib inline
np.random.seed(1)

%load_ext tensorboard

In [2]:
# # set reproducible results ???
# from numpy.random import seed
# seed(1)
# tf.random.set_seed(0)

In [3]:
# Loading the data (signs)
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

## Prepare the data


In [4]:
X_train_orig.shape, X_test_orig.shape, Y_train_orig.shape, Y_test_orig.shape

((1080, 64, 64, 3), (120, 64, 64, 3), (1, 1080), (1, 120))

In [5]:
x_train, x_test = X_train_orig / 255., X_test_orig / 255.
y_train, y_test = np.squeeze(Y_train_orig), np.squeeze(Y_test_orig)

## Baseline model

As a baseline we take the model from Coursera exercise.

Result: train_acc = 0.95, test_acc = 0.85 (once had train_acc = 0.98, test_acc = 0.9) - **add regularization.**

In [6]:
def create_baseline_model():
    """
    Specifies the architecture of our CNN. 
    return :: uncompiled model
    """
    inputs = keras.Input(shape=(64, 64, 3))

    C1 = layers.Conv2D(8, 4, activation='relu', strides=(1, 1), padding='same', 
                       kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(inputs) 
    
    P1 = layers.MaxPool2D(pool_size=(8, 8), strides=(8, 8), padding='same')(C1) 
    
    C2 = layers.Conv2D(16, 2, activation='relu', strides=(1, 1), padding='same',
                      kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(P1)
    
    P2 = layers.MaxPool2D(pool_size=(4, 4), strides=(4, 4), padding='same')(C2)

    F = layers.Flatten()(P2)

    outputs = layers.Dense(6, activation="softmax")(F)

    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model


def compile_baseline_model(model, learning_rate):
    """
    Specifies optimizer, loss function and evaluation metrics. 
    return :: compiled model
    """
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()]
    )
    
    return model


def get_summary(model):
    """
    Prints model's summary
    """
    model.summary()
    
    
def evaluate_model(model):
    """
    Evaluates model's performance
    """
    
    results = model.evaluate(x_train, y_train, batch_size=None)
    print("train loss, train acc:", results)

    results = model.evaluate(x_test, y_test, batch_size=None)
    print("test loss, test acc:", results)

In [7]:
baseline_model = create_baseline_model()
baseline_model = compile_baseline_model(baseline_model, learning_rate=0.001)

get_summary(baseline_model)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 8)         392       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 8)          0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          528       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 2, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 64)                0     

In [8]:
# Fit the model

BATCH_SIZE = 64
EPOCHS = 300

! rm -rf ./logs/fit/baseline/

log_dir = "./logs/fit/baseline/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = baseline_model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE, epochs=EPOCHS,
    callbacks=[tensorboard_callback]
)


Epoch 1/300


2022-09-07 13:51:35.944499: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17/17 [==============================] - 1s 20ms/step - loss: 1.8288 - sparse_categorical_accuracy: 0.1574
Epoch 2/300
17/17 [==============================] - 0s 27ms/step - loss: 1.7913 - sparse_categorical_accuracy: 0.1620
Epoch 3/300
17/17 [==============================] - 0s 28ms/step - loss: 1.7846 - sparse_categorical_accuracy: 0.2343
Epoch 4/300
17/17 [==============================] - 0s 28ms/step - loss: 1.7806 - sparse_categorical_accuracy: 0.1981
Epoch 5/300
17/17 [==============================] - 0s 28ms/step - loss: 1.7757 - sparse_categorical_accuracy: 0.2324
Epoch 6/300
17/17 [==============================] - 0s 29ms/step - loss: 1.7707 - sparse_categorical_accuracy: 0.3037
Epoch 7/300
17/17 [==============================] - 0s 26ms/step - loss: 1.7666 - sparse_categorical_accuracy: 0.2954
Epoch 8/300
17/17 [==============================] - 0s 28ms/step - loss: 1.7581 - sparse_categorical_accuracy: 0.3148
Epoch 9/300
17/17 [==============================] - 0s 28ms

17/17 [==============================] - 1s 30ms/step - loss: 0.6264 - sparse_categorical_accuracy: 0.7787
Epoch 70/300
17/17 [==============================] - 0s 28ms/step - loss: 0.6338 - sparse_categorical_accuracy: 0.7787
Epoch 71/300
17/17 [==============================] - 0s 29ms/step - loss: 0.6159 - sparse_categorical_accuracy: 0.7972
Epoch 72/300
17/17 [==============================] - 0s 28ms/step - loss: 0.6161 - sparse_categorical_accuracy: 0.7889
Epoch 73/300
17/17 [==============================] - 0s 29ms/step - loss: 0.6035 - sparse_categorical_accuracy: 0.8009
Epoch 74/300
17/17 [==============================] - 0s 28ms/step - loss: 0.5895 - sparse_categorical_accuracy: 0.7944
Epoch 75/300
17/17 [==============================] - 0s 28ms/step - loss: 0.5965 - sparse_categorical_accuracy: 0.7833
Epoch 76/300
17/17 [==============================] - 0s 27ms/step - loss: 0.5932 - sparse_categorical_accuracy: 0.7870
Epoch 77/300
17/17 [==============================] -

17/17 [==============================] - 0s 26ms/step - loss: 0.3794 - sparse_categorical_accuracy: 0.8815
Epoch 138/300
17/17 [==============================] - 0s 26ms/step - loss: 0.3758 - sparse_categorical_accuracy: 0.8694
Epoch 139/300
17/17 [==============================] - 0s 26ms/step - loss: 0.3791 - sparse_categorical_accuracy: 0.8731
Epoch 140/300
17/17 [==============================] - 0s 27ms/step - loss: 0.3906 - sparse_categorical_accuracy: 0.8750
Epoch 141/300
17/17 [==============================] - 0s 26ms/step - loss: 0.3806 - sparse_categorical_accuracy: 0.8685
Epoch 142/300
17/17 [==============================] - 0s 27ms/step - loss: 0.3672 - sparse_categorical_accuracy: 0.8787
Epoch 143/300
17/17 [==============================] - 0s 26ms/step - loss: 0.3697 - sparse_categorical_accuracy: 0.8787
Epoch 144/300
17/17 [==============================] - 0s 27ms/step - loss: 0.3628 - sparse_categorical_accuracy: 0.8898
Epoch 145/300
17/17 [=========================

17/17 [==============================] - 0s 26ms/step - loss: 0.2727 - sparse_categorical_accuracy: 0.9130
Epoch 205/300
17/17 [==============================] - 0s 27ms/step - loss: 0.2705 - sparse_categorical_accuracy: 0.9157
Epoch 206/300
17/17 [==============================] - 0s 26ms/step - loss: 0.2599 - sparse_categorical_accuracy: 0.9222
Epoch 207/300
17/17 [==============================] - 0s 28ms/step - loss: 0.2570 - sparse_categorical_accuracy: 0.9231
Epoch 208/300
17/17 [==============================] - 0s 26ms/step - loss: 0.2512 - sparse_categorical_accuracy: 0.9222
Epoch 209/300
17/17 [==============================] - 0s 27ms/step - loss: 0.2590 - sparse_categorical_accuracy: 0.9074
Epoch 210/300
17/17 [==============================] - 0s 27ms/step - loss: 0.2547 - sparse_categorical_accuracy: 0.9213
Epoch 211/300
17/17 [==============================] - 0s 27ms/step - loss: 0.2523 - sparse_categorical_accuracy: 0.9194
Epoch 212/300
17/17 [=========================

17/17 [==============================] - 0s 28ms/step - loss: 0.1996 - sparse_categorical_accuracy: 0.9333
Epoch 272/300
17/17 [==============================] - 0s 27ms/step - loss: 0.2006 - sparse_categorical_accuracy: 0.9398
Epoch 273/300
17/17 [==============================] - 0s 27ms/step - loss: 0.1861 - sparse_categorical_accuracy: 0.9435
Epoch 274/300
17/17 [==============================] - 0s 27ms/step - loss: 0.1839 - sparse_categorical_accuracy: 0.9463
Epoch 275/300
17/17 [==============================] - 0s 26ms/step - loss: 0.1775 - sparse_categorical_accuracy: 0.9528
Epoch 276/300
17/17 [==============================] - 0s 27ms/step - loss: 0.1755 - sparse_categorical_accuracy: 0.9463
Epoch 277/300
17/17 [==============================] - 0s 27ms/step - loss: 0.1775 - sparse_categorical_accuracy: 0.9519
Epoch 278/300
17/17 [==============================] - 0s 27ms/step - loss: 0.1775 - sparse_categorical_accuracy: 0.9500
Epoch 279/300
17/17 [=========================

In [9]:
%tensorboard --logdir logs/fit/baseline/

In [10]:
# Evaluate the model 
evaluate_model(baseline_model)

34/34 [==============================] - 0s 8ms/step - loss: 0.1595 - sparse_categorical_accuracy: 0.9574
train loss, train acc: [0.1594541370868683, 0.9574074149131775]
4/4 [==============================] - 0s 6ms/step - loss: 0.2895 - sparse_categorical_accuracy: 0.9083
test loss, test acc: [0.28954899311065674, 0.9083333611488342]


## Expand the dataset - data augmentation

Let us generate more data by flipping and 90 deg. rotating the original images.

In [11]:
import cv2

x_aug = [cv2.flip(img, 1) for img in x_train]
x_train_aug = np.append(x_train, x_aug, axis=0)
y_train_aug = np.append(y_train, y_train, axis=0)

x_aug = [cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE) for img in x_train]
x_train_aug = np.append(x_train_aug, x_aug, axis=0)
y_train_aug = np.append(y_train_aug, y_train, axis=0)

x_aug = [cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE) for img in x_train]
x_train_aug = np.append(x_train_aug, x_aug, axis=0)
y_train_aug = np.append(y_train_aug, y_train, axis=0)

ModuleNotFoundError: No module named 'cv2'

#### base-line model
Check whether we can improve the accuracy by adding more data.

Result: training is much slower, no improvement (probably should use **learning rate decay**).

In [ ]:
BATCH_SIZE = 32
EPOCHS = 500
LEARNING_RATE = 1e-3

baseline_model = create_baseline_model()
baseline_model = compile_baseline_model(baseline_model, learning_rate=LEARNING_RATE)

# Fit the model
! rm -rf ./logs/fit/baseline_aug/

log_dir = "./logs/fit/baseline_aug/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = baseline_model.fit(
    x_train_aug, y_train_aug,
    batch_size=BATCH_SIZE, epochs=EPOCHS,
    callbacks=[tensorboard_callback]
)


In [ ]:
%tensorboard --logdir logs/fit/baseline_aug/

In [ ]:
# Evaluate the model
evaluate_model(baseline_model)

## Final model

We use L2-regularization and Droupout layers to reduce overfitting. Learning rate decay is added for better optimization.

In [ ]:
def create_model(regularizer):
    """
    Specifies the architecture of our CNN. 
    return :: uncompiled model
    """
    inputs = keras.Input(shape=(64, 64, 3), name="signs")

    C1 = layers.Conv2D(32, 5, activation='relu', strides=(1, 1), padding='same', 
                      kernel_regularizer=regularizer)(inputs) 
    
    P1 = layers.MaxPool2D(pool_size=(4, 4), strides=(4, 4), padding='same')(C1)
    
    C2 = layers.Conv2D(64, 3, activation='relu', strides=(1, 1), padding='same',
                      kernel_regularizer=regularizer)(P1) 
    
    P2 = layers.MaxPool2D(pool_size=(4, 4), strides=(4, 4), padding='same')(C2) 

#     C3 = layers.Conv2D(64, 3, activation='relu', strides=(1, 1), padding='same',
#                       kernel_regularizer=regularizer)(P2) # CONV2D: stride 1, padding 'SAME'
    
    F = layers.Flatten()(P2)
    
    A1 = layers.Dense(120, activation="relu", 
                           kernel_regularizer=regularizer)(F)
    D1 = tf.keras.layers.Dropout(0.1)(A1)
    
    A2 = layers.Dense(84, activation="relu", 
                           kernel_regularizer=regularizer)(D1)
    D2 = tf.keras.layers.Dropout(0.1)(A2)

    outputs = layers.Dense(6, activation="softmax", name="predictions", 
                           kernel_regularizer=regularizer)(D2)

    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model


def compile_model(model, learning_rate):
    """
    Specifies optimizer, loss function and evaluation metrics. 
    return :: compiled model
    """
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()]
    )
    
    return model

#### original data

Result: train_acc = 0.98, test_acc = 0.95

In [ ]:
LEARNING_RATE = 1e-3
REGULARIZER = tf.keras.regularizers.L2(0.005)
BATCH_SIZE = 64
# BATCH_SIZE = x_train.shape[0]
EPOCHS = 100
REDUCE_LR_FACTOR = 0.5

model = create_model(regularizer=REGULARIZER)
model = compile_model(model, learning_rate=LEARNING_RATE)

# Fit the model
! rm -rf ./logs/fit/model
log_dir = "./logs/fit/model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# learning rate reduce by factor
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=REDUCE_LR_FACTOR,
                              patience=5, min_lr=1e-6) #monitor='val_loss'

history = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE, epochs=EPOCHS,
    validation_split=0.3,
    callbacks=[tensorboard_callback, reduce_lr]
)

In [ ]:
get_summary(model)


In [ ]:
%tensorboard --logdir logs/fit/model


In [ ]:
# Evaluate the model
evaluate_model(model)

#### augmented data

Result: is quite funny for val_acc vs. train_acc and test_acc. 
Why so?

In [ ]:
LEARNING_RATE = 1e-3
REGULARIZER = tf.keras.regularizers.L2(0.005)
BATCH_SIZE = 64
# BATCH_SIZE = x_train.shape[0]
EPOCHS = 150
REDUCE_LR_FACTOR = 0.5

model = create_model(regularizer=REGULARIZER)
model = compile_model(model, learning_rate=LEARNING_RATE)

# Fit the model
! rm -rf ./logs/fit/model
log_dir = "./logs/fit/model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# learning rate reduce by factor
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=REDUCE_LR_FACTOR,
                              patience=5, min_lr=1e-6) #monitor='val_loss'

history = model.fit(
    x_train_aug, y_train_aug,
    batch_size=BATCH_SIZE, epochs=EPOCHS,
    validation_split=0.3,
    callbacks=[tensorboard_callback, reduce_lr]
)

In [ ]:
%tensorboard --logdir logs/fit/model_aug/


In [ ]:
# Evaluate the model
evaluate_model(model)

## Debugging stuff

In [ ]:
# # Generate predictions
# predictions = model.predict(x_test)

# y_pred = np.argmax(predictions, axis = 1)

In [ ]:
# false_pred_index = [i for i,_ in enumerate(y_pred) if y_pred[i]!=y_test[i]]
# true_pred_index = [i for i,_ in enumerate(y_pred) if y_pred[i]==y_test[i]]
# false_pred_labels = [y_test[ind] for ind in false_pred_index]
# true_pred_labels = [y_test[ind] for ind in true_pred_index]

# print(f"True predicted labels: {np.unique(true_pred_labels, return_counts=True)}")
# print(f"False predicted labels: {np.unique(false_pred_labels, return_counts=True)}")

In [ ]:
# for index in false_pred_index:
#     print (f"i: {index}, pred: y = {y_pred[index]} \t true: y = {y_test[index]}")
#     plt.imshow(X_test_orig[index])
#     plt.show()

In [ ]:
# for index in true_pred_index:
#     print ("pred: y = " + str(y_pred[index]) + "\t" + "true: y = " + str(y_test[index]))
#     plt.imshow(X_test_orig[index])
#     plt.show()

In [ ]:
# for i in false_pred_index:
#     print(i, list(predictions[i]))
#     print(y_pred[i], y_test[i])